# Import and install Scene Object assets

This example notebook demonstrates how to create Scene Object assets from 3D asset files using the gRPC services.
This notebook can work from an empty solution or any example solution.

## Connect to solution

Let's start with the typical preamble:

- Import the relevant modules.
- Connect to the deployed solution.
- Store the solution grpc channel for later

In [ ]:
from intrinsic.solutions import deployments

solution = deployments.connect_to_selected_solution()
channel = solution.grpc_channel

## Connect to grpc services

We need the following grpc service connections to import a file and install it as a Scene Object asset.

- `scene_object_import` imports files as Intrinsic `SceneObject`s
- `installed_asset` installs `SceneObject` as assets in the solution
- `operations` serves the status of long running operations for both import and installation.

In [ ]:
from intrinsic.scene.proto.v1 import scene_object_import_pb2_grpc
from intrinsic.assets.proto import installed_assets_pb2_grpc
from google.longrunning import operations_pb2_grpc

scene_object_import_stub = scene_object_import_pb2_grpc.SceneObjectImportStub(channel)
installed_assets_stub = installed_assets_pb2_grpc.InstalledAssetsStub(channel)
operations_stub = operations_pb2_grpc.OperationsStub(channel)


## Long running operations utilities

Both the `SceneObjectImport` and `InstalledAsset` service returns longrunning operations.
Define some simple waiting mechanism to work with these operations.

In [ ]:
import time
from google.longrunning import operations_pb2

def wait_for_operation(operation: operations_pb2.Operation, operations_stub: operations_pb2_grpc.OperationsStub) -> operations_pb2.Operation:
  """Waits for an operation to complete.
  
  Raises RuntimeError if the operation fails.
  """
  while not operation.done:
    print(f"Waiting for operation {operation.name} to complete...")
    operation = operations_stub.GetOperation(
        operations_pb2.GetOperationRequest(name=operation.name)
    )
    time.sleep(1)
    print(f"Operation {operation.name} is not done yet.")
    
  if operation.HasField('error'):
    raise RuntimeError(f"Operation {operation.name} failed: {operation.error}")
  return operation


## Import a simple OBJ file as a Scene Object

This section demostrates the most basic use case, importing a single triangle OBJ file as an installed SceneObject asset

In [ ]:
from intrinsic.scene.proto.v1 import scene_object_import_pb2

# Just a triangle
simple_obj = """# Vertices
v 0.0 0.0 0.0
v 1.0 0.0 0.0
v 0.0 1.0 0.0
# Face (triangle) 
f 1 2 3
"""

# Prepare the import request
file = scene_object_import_pb2.SceneFileData(
    data=simple_obj.encode(),
    file_type=scene_object_import_pb2.SceneFileData.Type.OBJ,
)
request = scene_object_import_pb2.ImportSceneObjectRequest(
    file=file,
)

# Request and wait for operation done
operation = scene_object_import_stub.ImportSceneObject(request)
operation = wait_for_operation(operation, operations_stub)

# Unpack the response
response = scene_object_import_pb2.ImportSceneObjectResponse()
operation.response.Unpack(response)

# This is the imported Intrinsic SceneObject.
scene_object = response.scene_object
print(scene_object)


## Install the `SceneObject` as an asset

To utilize the `SceneObject` in a Flowstate solution, we need to install it as an asset. This section demostrates the installation steps.

In [ ]:
# Protos to construct the installation request
from intrinsic.assets.scene_objects.proto import scene_object_manifest_pb2
from intrinsic.assets.proto import documentation_pb2
from intrinsic.assets.proto import id_pb2
from intrinsic.assets.proto import installed_assets_pb2
from intrinsic.assets.proto import vendor_pb2

# Prepare installation requests
manifest = scene_object_manifest_pb2.ProcessedSceneObjectManifest(
              metadata=scene_object_manifest_pb2.SceneObjectMetadata(
                  id=id_pb2.Id(
                      name='new_object',
                      package='com.example',
                  ),
                  display_name='simple_triangle',
                  vendor=vendor_pb2.Vendor(
                      display_name='Example Vendor',
                  ),
                  documentation=documentation_pb2.Documentation(
                      description='Imported scene object',
                  ),
              ),
              assets=scene_object_manifest_pb2.ProcessedSceneObjectAssets(
                  scene_object_model=scene_object
              ),
          )

install_request = installed_assets_pb2.CreateInstalledAssetRequest(
    asset=installed_assets_pb2.CreateInstalledAssetRequest.Asset(
        scene_object=manifest
    ),
    policy=installed_assets_pb2.UpdatePolicy.UPDATE_POLICY_ADD_NEW_ONLY,
)

# Request and wait for operation done
operation = installed_assets_stub.CreateInstalledAsset(install_request)
operation = wait_for_operation(operation, operations_stub)

# Unpack the response
installed_asset = installed_assets_pb2.InstalledAsset()
operation.response.Unpack(installed_asset)

# Print the installed asset.
print(installed_asset)


# Import Configurations

This section introduces optional configurations for Scene Object import. Use `scene_object_import_pb2.ImportSceneObjectConfig` for setting import options that convert a file to a `SceneObject`.

The options are 
- Applying length unit conversion
- Removing collision geometry
- Transform imported Scene Object
- Geometry import resolution for CAD files with BRep geometry
- Material property overrides
- Appending custom opaque user data


### Define some utilities working with services

To simplify following demos, we first consolidate logic for calling `SceneObjectImport` and `InstalledAsset` grpc services for further use.

In [ ]:
from intrinsic.assets.scene_objects.proto import scene_object_manifest_pb2
from intrinsic.assets.proto import vendor_pb2
from intrinsic.assets.proto import documentation_pb2
from intrinsic.assets.proto import id_pb2
from intrinsic.scene.proto.v1 import scene_object_pb2
from intrinsic.scene.proto.v1 import scene_object_import_pb2
from intrinsic.scene.proto.v1 import scene_object_import_pb2_grpc
from intrinsic.assets.proto import installed_assets_pb2
from intrinsic.assets.proto import installed_assets_pb2_grpc
from google.longrunning import operations_pb2
from google.longrunning import operations_pb2_grpc

def import_scene_object(
    file_data: bytes,
    file_type: scene_object_import_pb2.SceneFileData.Type,
    config: scene_object_import_pb2.ImportSceneObjectConfig | None,
    scene_object_import_stub: scene_object_import_pb2_grpc.SceneObjectImportStub,
    operations_stub: operations_pb2_grpc.OperationsStub,
) -> scene_object_pb2.SceneObject:
    """
    Imports a scene object using the ImportSceneObject RPC.

    Args:
        file_data: The raw data of the scene file.
        file_type: The type of the scene file.
        config: Optional configuration for the import process.
        scene_object_import_stub: The gRPC stub for the SceneObjectImport service.
        operations_stub: The gRPC stub for the Operations service.

    Returns:
        The SceneObject imported.
    Raises:
        RuntimeError: If the operation fails.
    """
    file = scene_object_import_pb2.SceneFileData(
        data=file_data,
        file_type=file_type,
    )
    request = scene_object_import_pb2.ImportSceneObjectRequest(
        file=file,
        config=config,
    )

    operation = scene_object_import_stub.ImportSceneObject(request)
    operation = wait_for_operation(operation, operations_stub)

    response = scene_object_import_pb2.ImportSceneObjectResponse()
    operation.response.Unpack(response)
    return response.scene_object
  
def install_scene_object(
    scene_object: scene_object_pb2.SceneObject,
    name: str,
    installed_assets_stub: installed_assets_pb2_grpc.InstalledAssetsStub,
    operations_stub: operations_pb2_grpc.OperationsStub,
    update_policy: installed_assets_pb2.UpdatePolicy = installed_assets_pb2.UpdatePolicy.UPDATE_POLICY_ADD_NEW_ONLY,
) -> installed_assets_pb2.InstalledAsset:
    """
    Installs a scene object using the CreateInstalledAsset RPC.

    Args:
        scene_object: The scene object to install.
        name: The name of the scene object. Must be alpha-numeric with only underscores, starting with an alphabetic character.
        installed_assets_stub: The gRPC stub for the InstalledAssets service.
        operations_stub: The gRPC stub for the Operations service.
        update_policy: The update policy to use.

    Returns:
        The InstalledAsset.
    Raises:
        RuntimeError: If the operation fails.
    """
    request = installed_assets_pb2.CreateInstalledAssetRequest(
        asset=installed_assets_pb2.CreateInstalledAssetRequest.Asset(
            scene_object=scene_object_manifest_pb2.ProcessedSceneObjectManifest(
              metadata=scene_object_manifest_pb2.SceneObjectMetadata(
                  id=id_pb2.Id(
                      name=name,
                      package='com.example',
                  ),
                  display_name=name,
                  vendor=vendor_pb2.Vendor(
                      display_name='Example Vendor',
                  ),
                  documentation=documentation_pb2.Documentation(
                      description='Imported scene object',
                  ),
              ),
              assets=scene_object_manifest_pb2.ProcessedSceneObjectAssets(
                  scene_object_model=scene_object
              ),
          )
        ),
        policy=update_policy,
    )

    operation = installed_assets_stub.CreateInstalledAsset(request)
    operation = wait_for_operation(operation, operations_stub)    

    response = installed_assets_pb2.InstalledAsset()
    operation.response.Unpack(response)
    return response

### Apply material properties override

To override the material of the imported geometry, use the `material_properties` field to set a custom pbr material. The importer applies this material to all meshes in the import.

In [ ]:
from intrinsic.geometry.proto.v1 import material_pb2
from google.type import color_pb2
import uuid


config = scene_object_import_pb2.ImportSceneObjectConfig(
    # Changes the color to shiny red metal.
    material_properties= material_pb2.MaterialProperties(
        base_color=color_pb2.Color(
            red=1.0,
            green=0.0,
            blue=0.0
        ),
        metalness = 1.0,
        roughness = 0.0,
    ),
    scene_object_name = "object_with_material",
)
scene_object = import_scene_object(
    file_data = simple_obj.encode(),
    file_type = scene_object_import_pb2.SceneFileData.Type.OBJ,
    config= config,
    scene_object_import_stub=scene_object_import_stub,
    operations_stub = operations_stub,
)

installed_asset =install_scene_object(
    scene_object=scene_object, 
    name=f'so_with_material_{uuid.uuid4().hex}',
    installed_assets_stub=installed_assets_stub, 
    operations_stub=operations_stub
)
print(installed_asset.metadata.id_version)

### Append custom user data

To attach a custom user data field, pass in a `Any` protobuf message wrapping your custom data field. The data will be available in the `WorldObject`s created in the `ObjectWorld`.

In [ ]:
import uuid
from google.protobuf import any_pb2
from intrinsic.math.proto import vector3_pb2

my_vector = vector3_pb2.Vector3(x=1.0, y=2.0, z=3.0)
user_data_any = any_pb2.Any()
user_data_any.Pack(my_vector)


config = scene_object_import_pb2.ImportSceneObjectConfig(
    user_data = {'my_data': user_data_any},
    scene_object_name = "object_with_user_data",
)
scene_object = import_scene_object(
    file_data = simple_obj.encode(),
    file_type = scene_object_import_pb2.SceneFileData.Type.OBJ,
    config= config,
    scene_object_import_stub=scene_object_import_stub,
    operations_stub = operations_stub,
)

installed_asset =install_scene_object(
    scene_object=scene_object, 
    name=f'so_with_user_data_{uuid.uuid4().hex}',
    installed_assets_stub=installed_assets_stub, 
    operations_stub=operations_stub
)
print(installed_asset.metadata.id_version)

### Length unit conversion

Set `length_unit_conversion` option to uniformly scale imported geometry. By default Flowstate uses meters as length unit. This example demostrates adjusting units of the obj file authored as inches.

In [ ]:
import uuid

inch_to_m = 0.0254
config = scene_object_import_pb2.ImportSceneObjectConfig(
    length_unit_conversion=scene_object_import_pb2.LengthUnitConversion(
        scale_factor=inch_to_m,
    ),
    scene_object_name = "object_with_length_unit_conversion",
)

scene_object = import_scene_object(
    file_data = simple_obj.encode(),
    file_type = scene_object_import_pb2.SceneFileData.Type.OBJ,
    config= config,
    scene_object_import_stub=scene_object_import_stub,
    operations_stub = operations_stub,
)

installed_asset =install_scene_object(
    scene_object=scene_object, 
    name=f'so_with_length_unit_conversion_{uuid.uuid4().hex}',
    installed_assets_stub=installed_assets_stub, 
    operations_stub=operations_stub
)

print(installed_asset.metadata.id_version)

### Removing collision geometry

By default, imported objects have both visual("Intrinsic_Visual") and collision("Intrinsic_Collision") geometries generated. Use the `geometry_operations` field to remove collision geometry by specifying the `Intrinsic_Collision` type.

In [ ]:
config = scene_object_import_pb2.ImportSceneObjectConfig(
    # To remove the automatically generated collision meshes from the SceneObject
    geometry_operations=scene_object_import_pb2.GeometryOperations(
        remove_types = ["Intrinsic_Collision"]
    ),
    scene_object_name = "object_removed_collision",
)
scene_object = import_scene_object(
    file_data = simple_obj.encode(),
    file_type = scene_object_import_pb2.SceneFileData.Type.OBJ,
    config= config,
    scene_object_import_stub=scene_object_import_stub,
    operations_stub = operations_stub,
)

installed_asset =install_scene_object(
    scene_object=scene_object, 
    name=f'so_removed_collision_{uuid.uuid4().hex}',
    installed_assets_stub=installed_assets_stub, 
    operations_stub=operations_stub
)
print(installed_asset.metadata.id_version)

## View Mesh Statistcs

Obtain the aggregated mesh statistics for the visual and collision meshes on import

In [ ]:
from intrinsic.scene.proto.v1 import scene_object_import_pb2

# Just a triangle
simple_obj = """# Vertices
v 0.0 0.0 0.0
v 1.0 0.0 0.0
v 0.0 1.0 0.0
# Face (triangle) 
f 1 2 3
"""

# Prepare the import request
file = scene_object_import_pb2.SceneFileData(
    data=simple_obj.encode(),
    file_type=scene_object_import_pb2.SceneFileData.Type.OBJ,
)
request = scene_object_import_pb2.ImportSceneObjectRequest(
    file=file,
)

# Request and wait for operation done
operation = scene_object_import_stub.ImportSceneObject(request)
operation = wait_for_operation(operation, operations_stub)

# Unpack the response
response = scene_object_import_pb2.ImportSceneObjectResponse()
operation.response.Unpack(response)

visual_stats = response.visual_stats
print("Visual stats: ", visual_stats)
# By default, the collision mesh is the same as the visual mesh.
collision_stats = response.collision_stats
print("Collision stats: ", collision_stats)